<a href="https://colab.research.google.com/github/Sara-Rajaee/Deep_learning_explorations/blob/master/Copy_of_Copy_of_Copy_of_PT_Parameter_Efficient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [51]:
!git clone "https://github.com/Sara-Rajaee/adapter-transformers.git"

fatal: destination path 'adapter-transformers' already exists and is not an empty directory.


In [1]:
# @title Install Requirements
from IPython.display import clear_output
!pip install /content/adapter-transformers
#clear_output()

Processing ./adapter-transformers
  Created wheel for adapter-transformers: filename=adapter_transformers-2.11.0-cp36-none-any.whl size=692167 sha256=c1ecb22d83d0dd8b9e678b55c0f5ba9eac530d53c03fab16b75f46a7e9777133
  Stored in directory: /root/.cache/pip/wheels/69/f2/bd/f0720d145cbd02c42f9d833e06bb319e9ee518b24ebaa092e1
Successfully built adapter-transformers
  Found existing installation: adapter-transformers 2.11.0
    Uninstalling adapter-transformers-2.11.0:
      Successfully uninstalled adapter-transformers-2.11.0


In [2]:
# @title Import Requirements
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from tqdm.auto import tqdm, trange

from transformers import AutoModelWithHeads, AutoTokenizer, AdapterType
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from transformers.data.data_collator import DefaultDataCollator
from transformers import (
    glue_output_modes,
    glue_tasks_num_labels,
    glue_processors,
    glue_convert_examples_to_features,
)

import torch
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.utils.data.dataloader import DataLoader

if torch.cuda.is_available():    
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [3]:
# @title Hyperparameters

BATCH_SIZE =  32# @param {type:"integer"}
REDUCTION_FACTOR = 12#@param {type:"integer"}
NON_LINEARITY = "gelu" #@param ["gelu", "relu", "sewish"]
EPOCHS =  10#@param {type:"integer"}
MAX_LENGTH =   128#@param {type:"integer"}

TASK = "spam" #@param ["cola", "sst-2", "mrpc", "sts-b", "qqp", "mnli", "mnli-mm", "qnli", "rte", "wnli","spam"]

GLUE_DIR = "glue_datasets"
SAVED_ADAPTERS_DIR = "saved_adapters"
if not os.path.isdir(GLUE_DIR):
    os.mkdir(GLUE_DIR)
if not os.path.isdir(SAVED_ADAPTERS_DIR):
    os.mkdir(SAVED_ADAPTERS_DIR)
if not os.path.isdir(os.path.join(SAVED_ADAPTERS_DIR, TASK)):
    os.mkdir(os.path.join(SAVED_ADAPTERS_DIR, TASK))
    
if TASK == "mnli-m" or TASK == "mnli-mm":
  TASK_DIR = os.path.join(GLUE_DIR, "MNLI" )
elif TASK == "cola": 
  TASK_DIR = os.path.join(GLUE_DIR, "CoLA")
else:
  TASK_DIR = os.path.join(GLUE_DIR, TASK.upper())

num_labels = glue_tasks_num_labels["mnli" if "mnli" in TASK else TASK]
print(num_labels)
output_mode = glue_output_modes[TASK]
print(output_mode)
processor = glue_processors[TASK]()


LEARNING_RATE =  3e-4 #@param {type:"number"}
WARMUP_RATIO =   0.1 #@param {type:"number"}


2
classification


In [4]:
# @title Download GLUE data
""" Script for downloading all GLUE data.
Example usage:
    python download_glue_data.py --data_dir data --tasks all
Note: for legal reasons, we are unable to host MRPC.
You can either use the version hosted by the SentEval team, which is already tokenized,
or you can download the original data from:
https://download.microsoft.com/download/D/4/6/D46FF87A-F6B9-4252-AA8B-3604ED519838/MSRParaphraseCorpus.msi  # noqa
and extract the data from it manually.
For Windows users, you can run the .msi file. For Mac and Linux users, consider an external library
such as 'cabextract' (see below for an example). You should then rename and place specific files in
a folder (see below for an example).
mkdir MRPC
cabextract MSRParaphraseCorpus.msi -d MRPC
cat MRPC/_2DEC3DBE877E4DB192D17C0256E90F1D | tr -d $'\r' > MRPC/msr_paraphrase_train.txt
cat MRPC/_D7B391F9EAFF4B1B8BCE8F21B20B1B61 | tr -d $'\r' > MRPC/msr_paraphrase_test.txt
rm MRPC/_*
rm MSRParaphraseCorpus.msi
"""

import os
import sys
import shutil
import tempfile
import urllib.request
import zipfile
import pandas as pd

TASKS = ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "QNLI", "RTE", "WNLI","SPAM"]
TASK2PATH = {
    "CoLA": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FCoLA.zip?alt=media&token=46d5e637-3411-4188-bc44-5809b5bfb5f4",  # noqa
    "SST": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8",  # noqa
    "MRPC": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc",  # noqa
    "QQP": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQQP-clean.zip?alt=media&token=11a647cb-ecd3-49c9-9d31-79f8ca8fe277",  # noqa
    "STS": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSTS-B.zip?alt=media&token=bddb94a7-8706-4e0d-a694-1109e12273b5",  # noqa
    "MNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FMNLI.zip?alt=media&token=50329ea1-e339-40e2-809c-10c40afff3ce",  # noqa
    "SNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSNLIv2.zip?alt=media&token=6c15d9df-a597-47d1-9a43-0c605cbc6ef9",  # noqa
    "QNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FQNLIv2.zip?alt=media&token=6fdcf570-0fc5-4631-8456-9505272d1601",  # noqa
    "RTE": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FRTE.zip?alt=media&token=5efa7e85-a0bb-4f19-8ea2-9e1840f077fb",  # noqa
    "WNLI": "https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FWNLI.zip?alt=media&token=068ad0a0-ded7-4bd7-99a5-5e00222e0faf",  # noqa
    "SPAM": "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",
    "diagnostic": [
        "https://storage.googleapis.com/mtl-sentence-representations.appspot.com/tsvsWithoutLabels%2FAX.tsv?GoogleAccessId=firebase-adminsdk-0khhl@mtl-sentence-representations.iam.gserviceaccount.com&Expires=2498860800&Signature=DuQ2CSPt2Yfre0C%2BiISrVYrIFaZH1Lc7hBVZDD4ZyR7fZYOMNOUGpi8QxBmTNOrNPjR3z1cggo7WXFfrgECP6FBJSsURv8Ybrue8Ypt%2FTPxbuJ0Xc2FhDi%2BarnecCBFO77RSbfuz%2Bs95hRrYhTnByqu3U%2FYZPaj3tZt5QdfpH2IUROY8LiBXoXS46LE%2FgOQc%2FKN%2BA9SoscRDYsnxHfG0IjXGwHN%2Bf88q6hOmAxeNPx6moDulUF6XMUAaXCSFU%2BnRO2RDL9CapWxj%2BDl7syNyHhB7987hZ80B%2FwFkQ3MEs8auvt5XW1%2Bd4aCU7ytgM69r8JDCwibfhZxpaa4gd50QXQ%3D%3D",  # noqa
        "https://www.dropbox.com/s/ju7d95ifb072q9f/diagnostic-full.tsv?dl=1",
    ],
}

MRPC_TRAIN = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt"
MRPC_TEST = "https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt"


def download_and_extract(task, data_dir):
    print("Downloading and extracting %s..." % task)
    if task == "SPAM":
      data_file = task
      data_dir = data_dir + "/" + task
    else :
      data_file = "%s.zip" % task
    urllib.request.urlretrieve(TASK2PATH[task], data_file)
    with zipfile.ZipFile(data_file) as zip_ref:
        zip_ref.extractall(data_dir)
    os.remove(data_file)
    print("\tCompleted!")


def format_mrpc(data_dir, path_to_data):
    print("Processing MRPC...")
    mrpc_dir = os.path.join(data_dir, "MRPC")
    if not os.path.isdir(mrpc_dir):
        os.mkdir(mrpc_dir)
    if path_to_data:
        mrpc_train_file = os.path.join(path_to_data, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(path_to_data, "msr_paraphrase_test.txt")
    else:
        print("Local MRPC data not specified, downloading data from %s" % MRPC_TRAIN)
        mrpc_train_file = os.path.join(mrpc_dir, "msr_paraphrase_train.txt")
        mrpc_test_file = os.path.join(mrpc_dir, "msr_paraphrase_test.txt")
        urllib.request.urlretrieve(MRPC_TRAIN, mrpc_train_file)
        urllib.request.urlretrieve(MRPC_TEST, mrpc_test_file)
    assert os.path.isfile(mrpc_train_file), "Train data not found at %s" % mrpc_train_file
    assert os.path.isfile(mrpc_test_file), "Test data not found at %s" % mrpc_test_file
    urllib.request.urlretrieve(TASK2PATH["MRPC"], os.path.join(mrpc_dir, "dev_ids.tsv"))

    dev_ids = []
    with open(os.path.join(mrpc_dir, "dev_ids.tsv"), encoding="utf8") as ids_fh:
        for row in ids_fh:
            dev_ids.append(row.strip().split("\t"))

    with open(mrpc_train_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "train.tsv"), "w", encoding="utf8"
    ) as train_fh, open(os.path.join(mrpc_dir, "dev.tsv"), "w", encoding="utf8") as dev_fh:
        header = data_fh.readline()
        train_fh.write(header)
        dev_fh.write(header)
        for row in data_fh:
            label, id1, id2, s1, s2 = row.strip().split("\t")
            if [id1, id2] in dev_ids:
                dev_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))
            else:
                train_fh.write("%s\t%s\t%s\t%s\t%s\n" % (label, id1, id2, s1, s2))

    with open(mrpc_test_file, encoding="utf8") as data_fh, open(
        os.path.join(mrpc_dir, "test.tsv"), "w", encoding="utf8"
    ) as test_fh:
        header = data_fh.readline()
        test_fh.write("index\t#1 ID\t#2 ID\t#1 String\t#2 String\n")
        for idx, row in enumerate(data_fh):
            label, id1, id2, s1, s2 = row.strip().split("\t")
            test_fh.write("%d\t%s\t%s\t%s\t%s\n" % (idx, id1, id2, s1, s2))
    print("\tCompleted!")


def make_ready_example(task, spam_dir):
  if task == 'SPAM':
      df = pd.read_csv(spam_dir + '/SMSSpamCollection',sep = '\t',header = None, encoding = 'latin-1')
      df.columns = ["label", "text"]
      df.label = pd.factorize(df.label)[0]
      df.iloc[:4459].to_csv(spam_dir + '/train.tsv', sep='\t')
      df.iloc[4459:4459 + 557].to_csv(spam_dir + '/dev.tsv', sep='\t')
      df.iloc[4459 + 557:].to_csv(spam_dir + '/test.tsv', sep='\t')
  
if not os.path.isdir(GLUE_DIR):
    os.mkdir(GLUE_DIR)

spam_dir = os.path.join(GLUE_DIR, "SPAM")
if not os.path.isdir(spam_dir):
    os.mkdir(spam_dir)
for task in TASKS:
    if task == 'MRPC':
        format_mrpc(GLUE_DIR, "")
    else:
        download_and_extract(task, GLUE_DIR)
        if task == 'SPAM':
          make_ready_example(task, spam_dir)
#clear_output()
print("Completed")

	Completed!
	Completed!
Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!
Completed


In [65]:
!rm -r /content/glue_datasets/

In [62]:
df = pd.read_csv(spam_dir + '/SMSSpamCollection',sep = '\t',header = None, encoding = 'latin-1')
#df = df.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
df.columns = ["label", "text"]
df.label = pd.factorize(df.label)[0]
df.iloc[:4459].to_csv('dev.tsv', sep='\t')
df.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# @title Load Tokenizer, Model
pretrained_weights = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
model = AutoModelWithHeads.from_pretrained(pretrained_weights)
model.add_classification_head(TASK, num_labels=num_labels, layers=2)

model.add_adapter(adapter_name=TASK, adapter_type=AdapterType.text_task, config="houlsby")
model.train_adapter([TASK])
model.set_active_adapters(TASK)


In [6]:
# @title Example to Features
def example_to_features(mode, tokenizer, max_len, label_list, output_mode):
  if mode == "val":
    examples = processor.get_dev_examples(TASK_DIR)
  elif mode == "test":
    examples = processor.get_test_examples(TASK_DIR)
  elif mode == "train":
    examples = processor.get_train_examples(TASK_DIR)

  features = glue_convert_examples_to_features(
                      examples,
                      tokenizer,
                      max_length=max_len,
                      label_list=label_list,
                      output_mode=output_mode,
                  )
  return features

In [7]:
# @title Load Datasets
train_dataset = example_to_features("train", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)
val_dataset = example_to_features("val", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)
test_dataset = example_to_features("test", tokenizer, MAX_LENGTH, processor.get_labels(), output_mode)

data_collator = DefaultDataCollator()
train_dataloader = DataLoader(train_dataset, 
                              batch_size = BATCH_SIZE,
                              sampler = RandomSampler(train_dataset),
                              collate_fn=data_collator.collate_batch)
val_dataloader = DataLoader(
            val_dataset,
            batch_size = BATCH_SIZE,
            sampler = SequentialSampler(val_dataset),
            collate_fn=data_collator.collate_batch)

total_steps = len(train_dataloader) * EPOCHS

In [8]:
# @title Compute Metrics
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

def f1(preds, labels):
    return f1_score(y_true=labels, y_pred=preds)

def spearman(preds, labels):
    return spearmanr(preds, labels)[0]

def pearson(preds, labels):
    return pearsonr(preds, labels)[0]

def get_metricName(task_name):
    if task_name == "mrpc" or task_name == "qqp":
        return "f1"
    elif task_name == "cola":
        return "mcc"
    elif task_name == "sts-b":
        return "spearmanr"
    else:
        return "acc"

def glue_compute_metrics(metric_name, preds, labels):
    if output_mode == "classification":
        preds = np.argmax(preds, axis=1)
    elif output_mode == "regression":
        preds = np.squeeze(preds)

    assert len(preds) == len(labels)

    if metric_name == "f1":
        return f1(preds, labels)
    elif metric_name == "mcc":
        return matthews_corrcoef(labels, preds)
    elif metric_name == "spearmanr":
        return spearman(preds, labels)
    else:
        return simple_accuracy(preds, labels)

In [9]:
# @title Optimizer
optimizer = AdamW(model.parameters(), 
                  lr=LEARNING_RATE, 
                  eps=1e-6)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                num_warmup_steps=int(WARMUP_RATIO * total_steps),
                num_training_steps=total_steps)

In [10]:
# @title Train & Eval
metric_name = get_metricName(TASK)
history = {'train': {}, 'validation': {}}
history['train']['loss'] = []
history['validation']['loss'] = []
history['validation']['val_' + metric_name] = []
best_avg_val_loss = np.inf
best_avg_val_score = 0

model = model.to(device)
model.zero_grad()
train_iterator = trange(0, EPOCHS, desc="Epoch")
for epoch in train_iterator:

# Training
  model.train()
  train_loss = 0
  epoch_iterator = tqdm(train_dataloader, desc="Iteration")
  for step, inputs in enumerate(epoch_iterator):
    for k, v in inputs.items():
      inputs[k] = v.to(device)
    
    loss, logits = model(**inputs, adapter_names=[TASK])
    train_loss += loss.item()
    loss.backward()

    optimizer.step()
    scheduler.step()
    model.zero_grad()

  avg_train_loss = train_loss / len(train_dataloader) 
  print("Epoch: {0}".format(epoch+1))
  print("Average training loss: {0:.2f}".format(avg_train_loss))

# Evaluating
  model.eval()
  val_loss = 0
  val_score = 0
  for inputs in val_dataloader:
    for k, v in inputs.items():
      inputs[k] = v.to(device)
    
    with torch.no_grad(): 
      loss, logits = model(**inputs, adapter_names=[TASK])
    val_loss += loss.item()
    
    preds = logits.detach().cpu().numpy()
    label_ids = inputs["labels"].detach().cpu().numpy()
    val_score += glue_compute_metrics(metric_name, preds, label_ids)
  
  avg_val_loss = val_loss / len(val_dataloader) 
  avg_val_score = val_score / len(val_dataloader) 
  print("Average validation loss: {0:.2f}".format(avg_val_loss))
  print("Average validation {0}: {1:.2f}".format(metric_name, avg_val_score))

# Logg and save
  history['train']['loss'].append(avg_train_loss)
  history['validation']['loss'].append(avg_val_loss)
  history['validation']['val_' + metric_name].append(avg_val_score)
  
  if avg_val_score > best_avg_val_score or (avg_val_score == best_avg_val_score and avg_val_loss < best_avg_val_loss):
    best_avg_val_score = avg_val_score
    best_avg_val_loss = avg_val_loss
    
    model.save_adapter(os.path.join(SAVED_ADAPTERS_DIR, TASK, str(epoch+1)), TASK, with_head=True)
    print("Model saved as best model")



/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")



Epoch: 1
Average training loss: 0.42
Average validation loss: 0.39
Average validation acc: 0.87
Model saved as best model



Epoch: 2
Average training loss: 0.40
Average validation loss: 0.39
Average validation acc: 0.87



Epoch: 3
Average training loss: 0.40
Average validation loss: 0.39
Average validation acc: 0.87



Epoch: 4
Average training loss: 0.40
Average validation loss: 0.39
Average validation acc: 0.87


...


Epoch: 6
Average training loss: 0.39
Average validation loss: 0.40
Average validation acc: 0.87



Epoch: 7
Average training loss: 0.39
Average validation loss: 0.39
Average validation acc: 0.87



Epoch: 8
Average training loss: 0.39
Average validation loss: 0.41
Average validation acc: 0.87



Epoch: 9
Average training loss: 0.38
Average validation loss: 0.40
Average validation acc: 0.87



Epoch: 10
Average training loss: 0.38
Average validation loss: 0.40
Average validation acc: 0.87



# Save Model to Drive

In [ ]:
stop here

In [11]:
!mkdir /content/drive/My\ Drive/Saved\ Adapters/$TASK

In [12]:
!cp -r saved_adapters/$TASK/1/* /content/drive/My\ Drive/Saved\ Adapters/$TASK/

In [23]:
!git init


Reinitialized existing Git repository in /content/adapter-transformers/.git/


In [22]:
!git remote add origin https://<Sara-Rajaee>:<Sarah1374>@github.com/<Sara-Rajaee>/adapter-transformers.git

/bin/bash: Sara-Rajaee: No such file or directory


In [ ]:
!git clone "https://github.com/Sara-Rajaee/adapter-transformers.git"

fatal: destination path 'adapter-transformers' already exists and is not an empty directory.


In [ ]:
!git clone "https://github.com/Sara-Rajaee/adapter-transformers.git"

fatal: destination path 'adapter-transformers' already exists and is not an empty directory.


In [34]:
!git config --global user.email 'Sara.rajaei237@gmail.com'
!git config --global user.name "Sara-Rajaee"

from getpass import getpass
password = getpass('Password:')

Password:··········


In [32]:
%cd /content/adapter-transformers/

/content/adapter-transformers


In [35]:
!git add /content/adapter-transformers/src/transformers/data/processors/glue.py
!git commit -m 'add spam dataset'  # commit in Colab
!git push origin master

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
!git remote add origin https://<Sara-Rajaee>:<Sarah1374>github@github.com/<Sara-Rajaee>/reponame.git

In [27]:
!git push origin master

fatal: could not read Username for 'https://github.com': No such device or address
